In [4]:
import numpy as np
import cv2
import os
from sklearn.cluster import KMeans

In [2]:
input_dir='D:\Osteoporosis detection\datasets\Osteoporosis Knee X-ray\osteopenia'
processed_dir='D:\Osteoporosis detection\datasets\Osteoporosis Knee X-ray modified\Osteoporosis Knee X-ray\osteopenia'

In [15]:
dataset=[]
for img in os.listdir(input_dir):
    image=cv2.imread(os.path.join(input_dir,img))
    row,col,_=image.shape
    threshold=1
    max_brightness=0
    while threshold<col:
        left=image[:,:threshold,:]
        right=image[:,threshold:,:]
        break
    #kmeans=KMeans(n_clusters=2,n_init='auto').fit(image)
    break

In [14]:
image.s

3145728

In [22]:
import cv2
import numpy as np
import os


def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return binary

def connected_components_analysis(binary_image):
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary_image, connectivity=8)
    return num_labels, labels, stats, centroids

def filter_components_by_area(stats, min_area=1000):
    filtered_stats = []
    for i in range(1, len(stats)):  # Skipping the background component at index 0
        if stats[i, cv2.CC_STAT_AREA] > min_area:
            filtered_stats.append(stats[i])
    return filtered_stats

def save_image(image, filename, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    cv2.imwrite(os.path.join(output_dir, filename), image)

def separate_knees(image_path, output_dir):
    image = cv2.imread(image_path)
    binary = preprocess_image(image)
    num_labels, labels, stats, centroids = connected_components_analysis(binary)
    filtered_stats = filter_components_by_area(stats)
    
    if len(filtered_stats) == 1:
        # Only one knee detected, save the original image
        save_image(image, os.path.basename(image_path), output_dir)
    elif len(filtered_stats) >= 2:
        # Two or more knees detected, separate the two largest ones
        filtered_stats = sorted(filtered_stats, key=lambda x: x[cv2.CC_STAT_AREA], reverse=True)[:2]
        
        x1, y1, w1, h1 = filtered_stats[0][cv2.CC_STAT_LEFT], filtered_stats[0][cv2.CC_STAT_TOP], filtered_stats[0][cv2.CC_STAT_WIDTH], filtered_stats[0][cv2.CC_STAT_HEIGHT]
        x2, y2, w2, h2 = filtered_stats[1][cv2.CC_STAT_LEFT], filtered_stats[1][cv2.CC_STAT_TOP], filtered_stats[1][cv2.CC_STAT_WIDTH], filtered_stats[1][cv2.CC_STAT_HEIGHT]
        
        knee1 = image[y1:y1+h1, x1:x1+w1]
        knee2 = image[y2:y2+h2, x2:x2+w2]
        
        base_name = os.path.splitext(os.path.basename(image_path))[0]
        save_image(knee1, f"{base_name}_knee1.png", output_dir)
        save_image(knee2, f"{base_name}_knee2.png", output_dir)
    else:
        # No valid knees detected
        print(f"Unexpected number of knees detected in {image_path}")

def process_directory(input_dir, output_dir):
    for filename in os.listdir(input_dir):
        if filename.endswith((".png", ".jpg", ".jpeg")):
            image_path = os.path.join(input_dir, filename)
            separate_knees(image_path, output_dir)

# Example usage
input_dir='D:\Osteoporosis detection\datasets\Osteoporosis Knee X-ray\osteopenia'
processed_dir='D:\Osteoporosis detection\datasets\Osteoporosis Knee X-ray modified\Osteoporosis Knee X-ray\osteopenia'
process_directory(input_dir, processed_dir)

In [52]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def process_knee_xray(image_path,output_dir):
    # Step 1: Read the image
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Step 2: Apply a threshold to create a binary image
    #_, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    #print(binary)
    binary=(gray>0).astype(np.int8)
    #print(binary)
    # Step 3: Use connected components analysis to label different regions
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary, connectivity=8)
    
    # Step 4: Determine the regions corresponding to the knees
    knee_regions = []
    for i in range(1, num_labels):  # Start from 1 to skip the background
        area = stats[i, cv2.CC_STAT_AREA]
        if area > 1000:  # Assume knees are large regions
            knee_regions.append(i)
    
    if len(knee_regions) == 2:
        # Step 5: Split the image vertically between the knees
        knee_1 = labels == knee_regions[0]
        knee_2 = labels == knee_regions[1]
        
        # Find bounding boxes
        x1, y1, w1, h1 = stats[knee_regions[0], cv2.CC_STAT_LEFT], stats[knee_regions[0], cv2.CC_STAT_TOP], stats[knee_regions[0], cv2.CC_STAT_WIDTH], stats[knee_regions[0], cv2.CC_STAT_HEIGHT]
        x2, y2, w2, h2 = stats[knee_regions[1], cv2.CC_STAT_LEFT], stats[knee_regions[1], cv2.CC_STAT_TOP], stats[knee_regions[1], cv2.CC_STAT_WIDTH], stats[knee_regions[1], cv2.CC_STAT_HEIGHT]
        
        # Determine the middle point between the knees
        midpoint_x = (x1 + w1 + x2) // 2
        #print(x1,x2,w1,w2)
        # Split the image
        knee1_img = image[:, :midpoint_x]
        knee2_img = image[:, midpoint_x:]
        
        base_name = os.path.splitext(os.path.basename(image_path))[0]
        save_image(knee1_img, f"{base_name}_knee1.png", output_dir)
        save_image(knee2_img, f"{base_name}_knee2.png", output_dir)

    
    else:
        # Only one knee detected, save the original image
        save_image(image, os.path.basename(image_path), output_dir)

# Example usage
#process_knee_xray(os.path.join(input_dir,'OP2.jpeg'),processed_dir)

def process_directory(input_dir, output_dir):
    for filename in os.listdir(input_dir):
        image_path = os.path.join(input_dir, filename)
        process_knee_xray(image_path, output_dir)

# Example usage
input_dir='D:\Osteoporosis detection\datasets\Osteoporosis Knee X-ray\osteopenia'
processed_dir='D:\Osteoporosis detection\datasets\Osteoporosis Knee X-ray modified\Osteoporosis Knee X-ray\osteopenia'
process_directory(input_dir, processed_dir)

In [60]:
import cv2
import numpy as np
import os

def process_images(input_dir, output_dir, num_tests=5, threshold_start=127, threshold_step=10):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for filename in os.listdir(input_dir):
        if True:
            image_path = os.path.join(input_dir, filename)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if image is None:
                continue
            for i in range(num_tests):
                
                if True:
                    split_x=image.shape[1]//2

                    knee1 = image[:, :split_x]
                    knee2 = image[:, split_x:]
                    
                    # Save the split images
                    knee1_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_knee1.png")
                    knee2_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_knee2.png")
                    
                    cv2.imwrite(knee1_path, knee1)
                    cv2.imwrite(knee2_path, knee2)
                    break

# Define input and output directories
input_dir='D:\Osteoporosis detection\datasets\Osteoporosis Knee X-ray\osteopenia'
processed_dir='D:\Osteoporosis detection\datasets\Osteoporosis Knee X-ray modified\Osteoporosis Knee X-ray\osteopenia'

# Process the images
process_images(input_dir, processed_dir)
